In [1]:
!pip install -q chromadb sentence-transformers torch transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0

In [2]:
import os
import json
import time
from datetime import datetime
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings


Create Project Folder Structure

In [3]:
BASE_DIR = "/content/agent_memory_system"
LOG_DIR = f"{BASE_DIR}/logs"

os.makedirs(LOG_DIR, exist_ok=True)

print("Project directories created")


Project directories created


Initialize Embedding Model

In [4]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Initialize ChromaDB (Vector Memory Store)

In [5]:
chroma_client = chromadb.Client(
    Settings(
        persist_directory=f"{BASE_DIR}/chroma_db",
        anonymized_telemetry=False
    )
)

memory_collection = chroma_client.get_or_create_collection(
    name="agent_memory"
)


Define Logging Utility

In [29]:
from datetime import datetime, timezone

def log_event(event_type, data):
    log_entry = {
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "event_type": event_type,
        "data": data
    }
    with open(f"{LOG_DIR}/memory_events.jsonl", "a") as f:
        f.write(json.dumps(log_entry) + "\n")


Define Memory Scoring Functions

In [7]:
def calculate_recency_score(last_accessed):
    elapsed = time.time() - last_accessed
    return max(0, 1 - (elapsed / 3600))  # decays over 1 hour

def calculate_frequency_score(access_count):
    return min(1, access_count / 10)

def calculate_relevance_score(query_embedding, memory_embedding):
    import numpy as np
    return float(
        np.dot(query_embedding, memory_embedding) /
        (np.linalg.norm(query_embedding) * np.linalg.norm(memory_embedding))
    )


Memory Write Operation

In [23]:
def write_memory(text, memory_type="short_term"):
    embedding = embedding_model.encode(text).tolist()
    memory_id = f"mem_{int(time.time()*1000)}"

    metadata = {
        "memory_type": memory_type,
        "created_at": time.time(),
        "last_accessed": time.time(),
        "access_count": 0
    }

    memory_collection.add(
        documents=[text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[memory_id]
    )

    log_event("MEMORY_CREATED", {
        "id": memory_id,
        "type": memory_type,
        "content": text
    })


Memory Read Operation

In [24]:
def read_memory(query, top_k=1):
    query_embedding = embedding_model.encode(query).tolist()

    results = memory_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    for i, mem_id in enumerate(results["ids"][0]):
        meta = results["metadatas"][0][i]
        meta["access_count"] += 1
        meta["last_accessed"] = time.time()

        memory_collection.update(
            ids=[mem_id],
            metadatas=[meta]
        )

        log_event("MEMORY_RETRIEVED", {
            "id": mem_id,
            "content": results["documents"][0][i]
        })


Controlled Forgetting Mechanism

In [25]:
FORGET_THRESHOLD = 0.3

def forget_memories():
    all_data = memory_collection.get(
        include=["metadatas", "documents"]
    )

    for i, mem_id in enumerate(all_data["ids"]):
        meta = all_data["metadatas"][i]

        recency = calculate_recency_score(meta["last_accessed"])
        frequency = calculate_frequency_score(meta["access_count"])
        usefulness = (recency + frequency) / 2

        if usefulness < FORGET_THRESHOLD:
            memory_collection.delete(ids=[mem_id])

            log_event("MEMORY_FORGOTTEN", {
                "id": mem_id,
                "usefulness": usefulness,
                "content": all_data["documents"][i]
            })


Promote STM → LTM (Memory Lifecycle)

In [26]:
def promote_to_long_term(threshold=2):
    all_data = memory_collection.get(
        include=["metadatas", "documents"]
    )

    for i, mem_id in enumerate(all_data["ids"]):
        meta = all_data["metadatas"][i]

        if meta["access_count"] >= threshold and meta["memory_type"] != "long_term":
            meta["memory_type"] = "long_term"

            memory_collection.update(
                ids=[mem_id],
                metadatas=[meta]
            )

            log_event("MEMORY_PROMOTED", {
                "id": mem_id,
                "new_type": "long_term",
                "content": all_data["documents"][i]
            })


Demo: Full Memory Lifecycle

In [27]:
# Create memories
write_memory("User likes deep learning projects")
write_memory("User is working on agent memory systems")
write_memory("Temporary task note")

# Retrievals (force access_count ≥ 2)
read_memory("deep learning")
read_memory("deep learning")

read_memory("agent memory")
read_memory("agent memory")

read_memory("temporary note")
read_memory("temporary note")

# Promote
promote_to_long_term()

# Forget (optional but recommended)
forget_memories()


View Memory Logs

In [30]:
with open(f"{LOG_DIR}/memory_events.jsonl") as f:
    for line in f.readlines():
        print(line)


{"timestamp": "2026-02-03T07:00:04.580730", "event_type": "MEMORY_CREATED", "data": {"id": "mem_1770102004573", "type": "short_term", "content": "User likes deep learning projects"}}

{"timestamp": "2026-02-03T07:00:04.600917", "event_type": "MEMORY_CREATED", "data": {"id": "mem_1770102004597", "type": "short_term", "content": "User is working on agent memory systems"}}

{"timestamp": "2026-02-03T07:00:04.619061", "event_type": "MEMORY_CREATED", "data": {"id": "mem_1770102004615", "type": "short_term", "content": "Temporary task note"}}

{"timestamp": "2026-02-03T07:00:04.640111", "event_type": "MEMORY_RETRIEVED", "data": {"id": "mem_1770102004597", "content": "User is working on agent memory systems"}}

{"timestamp": "2026-02-03T07:00:04.643400", "event_type": "MEMORY_RETRIEVED", "data": {"id": "mem_1770102004615", "content": "Temporary task note"}}

{"timestamp": "2026-02-03T07:00:04.646513", "event_type": "MEMORY_RETRIEVED", "data": {"id": "mem_1770102004573", "content": "User likes

In [31]:
readme_content = """# Agent Memory & Forgetting System

This project implements an autonomous **agent memory lifecycle system** that can store, retrieve, promote, and manage memories over time.

The system demonstrates how an agent decides **what to remember**, **what to retain long-term**, and **when to promote information** based on usage.

---

## Key Features

- **Short-Term Memory (STM)**
  Temporary, task-specific memories created during each run.

- **Long-Term Memory (LTM)**
  Frequently accessed memories are automatically promoted and persist across executions.

- **Vector-Based Retrieval**
  Memories are embedded and retrieved using semantic similarity.

- **Autonomous Promotion Logic**
  Memories are promoted from STM to LTM based on access frequency (no manual intervention).

- **Persistent Memory Store**
  Memories created in earlier runs can be retrieved and promoted in later runs.

---

## Memory Lifecycle Events

The system logs all memory actions in JSON format:

- `MEMORY_CREATED` – A new memory is stored
- `MEMORY_RETRIEVED` – A memory is accessed based on relevance
- `MEMORY_PROMOTED` – A memory is promoted to long-term storage

These logs demonstrate realistic agent behavior across multiple executions.

---

## Notes

- Memory promotion is **usage-based**, not automatic
- Different runs may promote different memories
- This behavior is intentional and reflects real agent memory dynamics

---

## Execution

Run the notebook cells sequentially in Google Colab.
Logs are generated automatically during execution.
"""

with open("README.md", "w") as f:
    f.write(readme_content)

print("README.md file created successfully.")


README.md file created successfully.
